In [1]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as nr
import sys
import random
import time
import tensorflow as tf
from collections import deque
from keras.layers import Dense, Input, Add, GaussianNoise,Concatenate
from keras.optimizers import Adam, SGD, Nadam
from keras.models import Model
from keras import backend as K
from keras import regularizers

from mlagents.envs import UnityEnvironment

%matplotlib inline

print("Python version:")
print(sys.version)

# check Python version
if (sys.version_info[0] < 3):
    raise Exception("ERROR: ML-Agents Toolkit (v0.3 onwards) requires Python 3")

Python version:
3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]


Using TensorFlow backend.


In [2]:
env_name = "../env/CartPole"  # Name of the Unity environment binary to launch

In [3]:
env = UnityEnvironment(file_name=env_name)

# Set the default brain to work with
default_brain = env.brain_names[0]
brain = env.brains[default_brain]

INFO:mlagents.envs:
'CartPoleAcamedy' started successfully!
Unity Academy name: CartPoleAcamedy
        Number of Brains: 1
        Number of External Brains : 1
        Reset Parameters :
		
Unity brain name: CartPoleBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space size (per agent): 5
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): [1]
        Vector Action descriptions: 


In [4]:
class OU_noise():
    def __init__(self,action_size,mu=0,theta=0.03,sigma=0.1):
        self.action_size = action_size
        self.mu = mu
        self.theta = theta
        self.sigma = sigma
        self.state = np.ones(self.action_size)*self.mu
        #self.shape = np.shape(self.action_size)
        self.reset()

    def reset(self):
        self.state = np.ones(self.action_size)*self.mu

    def noise(self):
        x = self.state
        
        dx = self.theta * (self.mu - x) + self.sigma * nr.randn(self.action_size[0],self.action_size[1])
        self.state = x + dx
        #print(self.state)
        return self.state


class DDPGAgent:
    def __init__(self, state_size, agent_size,action_size):
        self.state_size = state_size
        self.agent_size = agent_size
        self.action_size = action_size
        self.load_model = False
        self.Gausian_size = 0.01
        self.gard_clip_radious = 100.0

        # build networks
        self.actor = self.build_actor()
        self.actor_target = self.build_actor()
        self.critic = self.build_critic()
        self.critic_target = self.build_critic()
        self.actor_updater = self.actor_optimizer()

        self.memory = deque(maxlen=50000)
        self.batch_size = 64
        self.discount_factor = 0.99
        self.epsilon = 1
        self.epsilon_decay = 0.99999
        
        self.noiser = OU_noise([agent_size,self.action_size]) #수정한 부분
        
        if self.load_model:
            self.actor.load_weights("3DBall_actor.h5")
            self.actor_target.load_weights("3DBall_actor.h5")
            self.critic.load_weights("3DBall_critic.h5")
            self.critic_target.load_weights("3DBall_critic.h5")

    def build_actor(self):
        print("building actor network")
        input = Input(shape=[self.state_size])
        h1 = Dense(64, activation='elu')(input)
        h1 = Dense(64, activation='elu')(h1)
        h1 = Dense(64, activation='elu')(h1)
        h1 = Dense(64, activation='elu')(h1)
        h1 = Dense(64, activation='elu')(h1)
        h1 = Dense(64, activation='elu')(h1)
        h1 = Dense(64, activation='elu')(h1)
        action = Dense(self.action_size, activation='tanh')(h1)
        actor = Model(inputs=input, outputs=action)
        actor.summary()
        return actor

    def actor_optimizer(self):
        actions = self.actor.output
        dqda = tf.gradients(self.critic.output, self.critic.input)
        loss = actions * tf.clip_by_value(-dqda[1],-self.gard_clip_radious,self.gard_clip_radious) #fit in 50

        optimizer = Adam(lr=0.00001)
        updates = optimizer.get_updates(self.actor.trainable_weights, [], loss)
        train = K.function([self.actor.input, self.critic.input[0],
                            self.critic.input[1]], [], updates=updates)
        return train

    def build_critic(self):
        print("building critic network")
        state = Input(shape=[self.state_size], name='state_input')
        action = Input(shape=[self.action_size], name='action_input')
        w1 = Dense(64, activation='elu')(state)
        w1 = Dense(64, activation='elu')(w1)
        a1 = Dense(64, activation='elu')(action)
        a1 = Dense(64, activation='elu')(a1)
        c = Concatenate()([w1,a1])
        #c = Add()([w1,a1])
        h2 = Dense(64, activation='elu')(c)
        h2 = Dense(64, activation='elu')(h2)
        h2 = Dense(64, activation='elu')(h2)
        h2 = Dense(64, activation='elu')(h2)
        h2 = Dense(64, activation='elu')(h2)
        h2 = Dense(64, activation='elu')(h2)
        h2 = Dense(64, activation='elu')(h2)
        Velue = Dense(1, activation='linear')(h2)
        critic = Model(inputs=[state, action], outputs=Velue)
        critic.compile(loss='mse', optimizer=Adam(lr=0.00001))
        critic.summary()
        return critic

    def get_action(self, state):
        self.epsilon = max(self.epsilon*self.epsilon_decay,0.05)
        #print(state)
        action = self.actor.predict(state)

        real = action + self.epsilon*self.noiser.noise()
        return np.clip(real,-1.1,1.1)
    
    def gat_action_nonoise(self,state):
        action = self.actor.predict(state)
        
        real = action + 0.1*self.noiser.noise()
        return np.clip(real,-1.1,1.1)

    def append_sample(self, state, action, reward, next_state, done):
        
        for i in range(self.agent_size):
            self.memory.append((state[i], action[i], reward[i], next_state[i], done[i]))
        #self.memory.append((state, action, reward, next_state, done))

    def train_model(self):
        # make mini-batch from replay memory
        mini_batch = random.sample(self.memory, self.batch_size)

        states = np.asarray([e[0] for e in mini_batch])
        actions = np.asarray([e[1] for e in mini_batch])
        rewards = np.asarray([e[2] for e in mini_batch])
        next_states = np.asarray([e[3] for e in mini_batch])
        dones = np.asarray([e[4] for e in mini_batch])

        # update critic network
        critic_action_input = self.actor_target.predict(next_states)
        target_q_values = self.critic_target.predict([next_states, critic_action_input])

        targets = np.zeros([self.batch_size, 1])
        for i in range(self.batch_size):
            if dones[i]:
                targets[i] = rewards[i]
            else:
                targets[i] = rewards[i] + self.discount_factor * target_q_values[i]

        self.critic.train_on_batch([states, actions], targets)

        # update actor network
        a_for_grad = self.actor.predict(states)
        self.actor_updater([states, states, a_for_grad])
        #self.actor_updater([states, states, actions])
        
    def train_critic(self):
        mini_batch = random.sample(self.memory, self.batch_size)

        states = np.asarray([e[0] for e in mini_batch])
        actions = np.asarray([e[1] for e in mini_batch])
        rewards = np.asarray([e[2] for e in mini_batch])
        next_states = np.asarray([e[3] for e in mini_batch])
        dones = np.asarray([e[4] for e in mini_batch])
        
        self.critic.train_on_batch([states, actions], rewards)

    # after some time interval update the target model to be same with model
    def update_target_model(self):
        self.actor_target.set_weights(self.actor.get_weights())
        self.critic_target.set_weights(self.critic.get_weights())

In [5]:
# Reset the environment
env_info = env.reset(train_mode=True)[default_brain]

# Examine the state space for the default brain
print("Agent state looks like: \n{}".format(env_info.vector_observations[0]))

print("Agent shape looks like: \n{}".format(np.shape(env_info.vector_observations)))

for observation in env_info.visual_observations:
    print("Agent observations look like:")
    if observation.shape[3] == 3:
        plt.imshow(observation[0,:,:,:])
    else:
        plt.imshow(observation[0,:,:,0])
print("Agent shape looks like: \n{}".format(np.shape(env_info.visual_observations)))

agent = DDPGAgent(5,10, 1)

Agent state looks like: 
[-0.08413965  0.50443733  0.          3.37213826  0.        ]
Agent shape looks like: 
(10, 5)
Agent shape looks like: 
(0,)
building actor network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                384       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None,

In [ ]:
train_mode =  True  # Whether to run the environment in training or inference mode

reward_memory = deque(maxlen=20)
agents_reward = np.zeros(agent.agent_size)
env_info = env.reset(train_mode=train_mode)[default_brain]
for episode in range(10000000):
    #env_info = env.reset(train_mode=train_mode)[default_brain]
    state = env_info.vector_observations
    done = False
    episode_rewards = 0
    for i in range(100):
        
        if not train_mode:
            action = agent.get_action(state)
        else:
            action = agent.get_action(state)
        #action = np.column_stack([np.random.randint(0, action_size[i], size=(len(env_info.agents))) for i in range(len(action_size))])
        env_info = env.step(action)[default_brain]
        next_state = env_info.vector_observations
        reward = env_info.rewards
        agents_reward += reward
        #episode_rewards += reward#env_info.rewards[0]
        done = env_info.local_done
        for idx,don in enumerate(done):
            if don:
                reward_memory.append(agents_reward[idx])
                agents_reward[idx] = 0
        if train_mode:
            agent.append_sample(state,action,reward,next_state,done)
        state = next_state
        
        if len(agent.memory) > agent.batch_size and train_mode:
            agent.train_model()
        
    #agent.noiser.reset()
    agent.update_target_model()
    if agent.epsilon <= 0.05:
        agent.epsilon = 0.99

    if episode%50 == 0 and not episode == 0 and train_mode:
        
        agent.actor.save("3DBall_actor.h5")
        agent.critic.save("3DBall_critic.h5")
        print("model saved")
    print("episode_{} reward: {} episilon: {}".format(episode,np.mean(reward_memory),agent.epsilon))

episode_0 reward: 10.479934573173523 episilon: 0.9990004948383437
episode_1 reward: 7.9904765605926515 episilon: 0.9980019886872552
episode_2 reward: 4.530595290660858 episilon: 0.9970044805482192
episode_3 reward: 4.48782849907875 episilon: 0.9960079694237177
episode_4 reward: 4.250889068841934 episilon: 0.9950124543172282
episode_5 reward: 4.164966660737991 episilon: 0.994017934233226
episode_6 reward: 4.645557117462158 episilon: 0.9930244081771812
episode_7 reward: 5.114448535442352 episilon: 0.9920318751555576
episode_8 reward: 3.975300818681717 episilon: 0.9910403341758119
episode_9 reward: 3.776609319448471 episilon: 0.9900497842463938
episode_10 reward: 3.7959738254547117 episilon: 0.9890602243767426
episode_11 reward: 4.625797080993652 episilon: 0.9880716535772887
episode_12 reward: 4.52214515209198 episilon: 0.9870840708594518
episode_13 reward: 4.1273159146308895 episilon: 0.9860974752356391
episode_14 reward: 3.5458719551563265 episilon: 0.9851118657192448
episode_15 reward:

episode_124 reward: 79.30319168567658 episilon: 0.8824963510210262
episode_125 reward: 75.67915295362472 episilon: 0.8816142913630383
episode_126 reward: 106.38317249417305 episilon: 0.8807331133282309
episode_127 reward: 74.30730329751968 episilon: 0.8798528160354181
episode_128 reward: 66.67981781363487 episilon: 0.8789733986042928
episode_129 reward: 84.30796032547951 episilon: 0.878094860155429
episode_130 reward: 94.7100695848465 episilon: 0.8772171998102793
episode_131 reward: 90.92554764151573 episilon: 0.8763404166911752
episode_132 reward: 104.68920612931251 episilon: 0.8754645099213243
episode_133 reward: 89.08413852453232 episilon: 0.8745894786248111
episode_134 reward: 85.58658061623574 episilon: 0.8737153219265954
episode_135 reward: 99.37981079220772 episilon: 0.8728420389525116
episode_136 reward: 104.57235450744629 episilon: 0.8719696288292675


In [11]:
print(reward)

[1.0120590925216675, 1.0189833641052246, 1.0423592329025269, 1.014908790588379, 1.0127912759780884, 1.2123252153396606, 1.0106924772262573, 1.0095158815383911, 1.0150420665740967, 1.0495394468307495]


In [ ]:
agent.epsilon_decay =0.999